# Imports

In [1]:
# imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn import preprocessing

# Load
This is where the prepared data is loaded in for use in training and testing of the model

In [2]:
# Load cleaned and preprocessed data
X_train = pd.read_csv("data/X_train.csv", index_col=0).astype(float)
y_train = pd.read_csv("data/y_train.csv", index_col=0).astype(int)
y_train = pd.Series(y_train.values.ravel())
y_train.index = X_train.index
y_train_binary = pd.read_csv("data/y_train_binary.csv", index_col=0).astype(int)
y_train_binary = pd.Series(y_train_binary.values.ravel())
y_train_binary.index = X_train.index
X_test = pd.read_csv("data/X_test.csv", index_col=0).astype(float)
y_test = pd.read_csv("data/y_test.csv", index_col=0).astype(int)
y_test = pd.Series(y_test.values.ravel())
y_test.index = X_test.index
y_test_binary = pd.read_csv("data/y_test_binary.csv", index_col=0).astype(int)
y_test_binary = pd.Series(y_test_binary.values.ravel())
y_test_binary.index = X_test.index

# Two Stage SVM RBF Model
This section is where the two stage SVM RBF model is trained and tested. First class 1 and class 2 are merged into a single class and a SVM RBF model is trained to predict class 0 against the combined class 1 and 2. A second model is then trained to predict class 1 against class 2. The output of these two models is then combined to create a prediction of all classes

In [3]:
#Model parameters
desired_c = 1
desired_gamma = 0.01

# Stage 1: SVM RBF - 0 vs (1+2)
svm_stage1 = SVC(kernel='rbf', gamma=desired_gamma, C=desired_c, probability=True, class_weight='balanced')
svm_stage1.fit(X_train, y_train_binary)

y_pred_binary = svm_stage1.predict(X_test)
y_prob_binary = svm_stage1.predict_proba(X_test)

print("\nStage 1 SVM Classification Report (0 vs 1+2):")
print(classification_report(y_test_binary, y_pred_binary))

# Stage 2: SVM RBF - 1 vs 2
# Get original test labels with 0/1/2
y_test_full = y_test.loc[y_test_binary.index]

# Find samples predicted as diabetic
indices_pred_diabetes = np.where(y_pred_binary == 1)[0]
X_test_diabetes = X_test.iloc[indices_pred_diabetes]
y_test_diabetes = y_test_full.iloc[indices_pred_diabetes]

# Keep only class 1 and 2
# Pull records where y_test_diabetes is class 1 or class 2, along with their indexes
test_diabetes = y_test_diabetes.loc[(y_test_diabetes == 1) | (y_test_diabetes == 2)].index
X_test_diabetes = X_test_diabetes.loc[test_diabetes]
y_test_diabetes = y_test_diabetes.loc[test_diabetes]

# Prepare second-stage training data
# Get the indexes where y_train_binary == 1
diabetes_indexes = y_train_binary.loc[y_train_binary == 1].index

# Use these indexes to select records from X_train_diabetes and y_train_diabetes
X_train_diabetes = X_train.loc[diabetes_indexes]
y_train_diabetes = y_train.loc[diabetes_indexes]

# Train second SVM
svm_stage2 = SVC(kernel='rbf', gamma=desired_gamma, C=desired_c, probability=True, class_weight='balanced')
svm_stage2.fit(X_train_diabetes, y_train_diabetes)

y_pred_second_stage = svm_stage2.predict(X_test_diabetes)
y_prob_second_stage = svm_stage2.predict_proba(X_test_diabetes)

print("\n\nStage 2 SVM Classification Report (1 vs 2):")
print(classification_report(y_test_diabetes, y_pred_second_stage))

# Reconstruct final prediction array
y_pred_final = y_pred_binary.copy()
pred_diabetes_indices = np.where(y_pred_binary == 1)[0]

# Reindex and assign
y_test_full_diabetes = y_test_full.iloc[pred_diabetes_indices]
valid_indices = pred_diabetes_indices[(y_test_full_diabetes == 1) | (y_test_full_diabetes == 2)]

for i, idx in enumerate(valid_indices):
    y_pred_final[idx] = y_pred_second_stage[i]

print("\nFinal SVM Classification Report:")
print(classification_report(y_test_full, y_pred_final))


Stage 1 SVM Classification Report (0 vs 1+2):
              precision    recall  f1-score   support

           0       0.94      0.66      0.78     38012
           1       0.33      0.81      0.47      7945

    accuracy                           0.69     45957
   macro avg       0.64      0.73      0.62     45957
weighted avg       0.84      0.69      0.72     45957



Stage 2 SVM Classification Report (1 vs 2):
              precision    recall  f1-score   support

           1       0.16      0.33      0.21       653
           2       0.91      0.80      0.85      5752

    accuracy                           0.75      6405
   macro avg       0.54      0.56      0.53      6405
weighted avg       0.84      0.75      0.79      6405


Final SVM Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.66      0.78     38012
           1       0.02      0.23      0.03       926
           2       0.91      0.66      0.76      7019

 

# Export
This section is where the final predictions of the two stage SVM RBF model are exported for use by the ensemble

In [4]:
# Save final predictions
# --------------------------------------------
import os
os.makedirs("results", exist_ok=True)
np.save("results/y_pred_svm.npy", y_pred_final)